In [1]:
import sys
import argparse
import collections
from scipy.optimize import linprog
ERROR = 1e-10

In [2]:
#Write a function that computes f(r)
#f(r) is created to construct the YTM of the bond
#Note that in this equation,  all cash flows are discounted at the same rate, the ytm that we are trying to solve
def my_function(cash_flow, price, rate) :
    result = price * pow((1+rate), len(cash_flow))
    for i in range(len(cash_flow)) :
        result = result - (cash_flow[i] * pow((1+rate), len(cash_flow)-1-i))
    return result

In [3]:
# write a function that computes f'(r) which is an inportant element in Newton-Raphson method
def derivative_function(cash_flow, price, rate) :
    result = len(cash_flow) * price * pow((1+rate), len(cash_flow)-1)
    for i in range(len(cash_flow)-1) :
        result = result - ((len(cash_flow) - 1 - i) * cash_flow[i] * pow((1+rate), (len(cash_flow)-2-i)))
    return result

In [4]:
# write a function that finds the (only) root of f(r)
# using Newton-Raphson method
def Newton_Raphson(cash_flow, price, rate) :
    while (abs(my_function(cash_flow, price, rate)) > ERROR) :
        rate = rate - my_function(cash_flow, price, rate)/derivative_function(cash_flow, price, rate)
    return rate

In [5]:
#write a function that computes the duration of a cash flow
def get_duration(cash_flow, price, rate) :
    duration = 0.0
    for i in range(len(cash_flow)) :
        duration = duration + (cash_flow[i] *(i+1))/pow((1+rate), i+1)
    duration = duration/price
    return duration

In [6]:
#write a function that computes the convexity of a cash flow
def get_convexity(cash_flow, price, rate) :
    convexity = 0.0
    for i in range(len(cash_flow)) :
        convexity = convexity + (cash_flow[i]*((i+1)*(i+2))/pow((1+rate), i+3))
    convexity = convexity/price
    return (convexity)

In [7]:
def print_data(filename) :

    global debt_obligation_amount
    global time_when_debt_is_due
    global number_of_cash_flows
    global price_list
    
    print ("Input File: ", filename)
    print ("We owe " + str(debt_obligation_amount) + " in " + str(time_when_debt_is_due) + " years")
    print ("Number of Cash Flows: " + str(number_of_cash_flows))
    for i in range(number_of_cash_flows):
        print ("------------------------------------------------------------------")
        print ("Cash Flow #" + str(i+1))
        print ("Price = " + str(price_list[i]))
        print ("Maturity = " + str(maturity_list[i]))
        print ("Yield to Maturity = " + str(yield_to_maturity[i]))
        print ("Duration = " + str(duration[i]))
        print ("Convexity = " + str(convexity[i]))

In [11]:
def get_data(file_name) :
    global price_list
    global duration
    global convexity
    global maturity_list
    global time_when_debt_is_due
    global yield_to_maturity
    global number_of_cash_flows
    global debt_obligation_amount
    global cash_flow
    
    with open(file_name) as f:
        debt_obligation_amount, time_when_debt_is_due = [float(x) for x in next(f).split()] # read first line
        number_of_cash_flows= int(f.readline())
        array = []
        for line in f: # read rest of lines
            array.append([float(x) for x in line.split()])
        
    price_list = collections.deque()
    maturity_list = collections.deque()
    cash_flow = collections.deque()
    
    for i in range(number_of_cash_flows) :
        price_list.append(array[i][0])
        maturity_list.append(int(array[i][1]))
        X = collections.deque()
        for j in range(2,maturity_list[i]+2) :
            X.append(array[i][j])
        cash_flow.append(X)
    
    duration = collections.deque()
    convexity = collections.deque()
    yield_to_maturity = collections.deque()
    
    for i in range(number_of_cash_flows) :
        r = Newton_Raphson(cash_flow[i], price_list[i], 0.0)
        yield_to_maturity.append(r)
        d = get_duration(cash_flow[i], price_list[i], r)
        duration.append(d)
        c = get_convexity(cash_flow[i], price_list[i], r)
        convexity.append(c)
        

In [12]:
def get_optimal_portfolio():
    # write the lp_solve function in Python that computes the optimal_portfolio
    # for details on linprog see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
    
    global number_of_cash_flows
    global duration
    global time_when_debt_is_due
    
    # getting the cost function that needs to be minimized
    cost = [0 for i in range(number_of_cash_flows)]
    for i in range(number_of_cash_flows) :
        cost[i] = -1.0 * convexity[i]
    # scipy.linprog minimizes, hence the multiplication by -1
    
    A = [ [0 for i in range(number_of_cash_flows)] for j in range(2) ]
    for i in range(number_of_cash_flows) :
        A[0][i] = 1.0
        A[1][i] = duration[i]
 
    b = [1 for i in range(2)]
    b[1] = time_when_debt_is_due
    
    res = linprog(cost, A_eq=A, b_eq=b)
    
    if (res.success == True) :
        print ("------------------------------------------------------------------")
        print ("The largest convexity we can get is : " + str(-1*res.fun))
        print ("The optimal portfolio: ")
        answer = res.x
        for i in range(number_of_cash_flows) :
            print ("%Cash Flow: " + str(i+1) + ": " + str(answer[i]))
        
        print ("\nTo immunize against small changes in `r' for each $1 of PV, you should buy")
        for i in range(number_of_cash_flows) :
            if (answer[i] != 0) : 
                print ("$" + str(answer[i]) + " of Cash Flow# " + str(i+1))
        print ("\nNote: You have to figure out a (reasonable) way of estimating the PV for a \nFuture Debt Obligation when `r' is expected to change")
        print ("------------------------------------------------------------------")
    else :
        print ("------------------------------------------------------------------")
        print ("There is no portfolio that meets the duration constraint of " + str(time_when_debt_is_due) + " years")

In [13]:
filename = "input1"
get_data(filename)
print_data(filename)
get_optimal_portfolio()

Input File:  input1
We owe 1790.85 in 10.0 years
Number of Cash Flows: 5
------------------------------------------------------------------
Cash Flow #1
Price = 1131.27
Maturity = 10
Yield to Maturity = 0.04999993944363203
Duration = 7.75869645880194
Convexity = 70.42635827407678
------------------------------------------------------------------
Cash Flow #2
Price = 1069.88
Maturity = 15
Yield to Maturity = 0.06256394215391257
Duration = 9.935820137349099
Convexity = 119.83139286043489
------------------------------------------------------------------
Cash Flow #3
Price = 863.5
Maturity = 30
Yield to Maturity = 0.07000004955515875
Duration = 13.677444200173408
Convexity = 262.76902615152306
------------------------------------------------------------------
Cash Flow #4
Price = 1148.75
Maturity = 12
Yield to Maturity = 0.057499916729924286
Duration = 8.580822815473859
Convexity = 87.67983713567897
------------------------------------------------------------------
Cash Flow #5
Price = 11